In [2]:
from langchain.llms import GooglePalm
import os

api_key = os.environ('GOOGLE_API_KEY')
llm = GooglePalm(google_api_key = api_key, temperature = 0.4)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = "Arka%40281004"  # Updated password
db_host = "localhost:3306"
db_name = "db_tshirts"

# Correctly format the URI, specifying the database driver (pymysql)
db_uri = f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}"

# Create an instance of SQLDatabase
db = SQLDatabase.from_uri(db_uri, sample_rows_in_table_info=3)

# Access table information
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [8]:
from langchain_experimental.sql import SQLDatabaseChain

In [9]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(34,)]
Answer:34
> Finished chain.


### Above is correct query 👍

In [10]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('17927'),)]
Answer:17927
> Finished chain.


### Above is correct query 👍

In [11]:
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(price - (price * pct_discount)) AS revenue
FROM t_shirts AS t
JOIN discounts AS d ON t.t_shirt_id = d.t_shirt_id
WHERE t.brand = 'Levi' AND CURDATE() BETWEEN d.start_date AND d.end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'd.start_date' in 'where clause'")
[SQL: SELECT SUM(price - (price * pct_discount)) AS revenue
FROM t_shirts AS t
JOIN discounts AS d ON t.t_shirt_id = d.t_shirt_id
WHERE t.brand = 'Levi' AND CURDATE() BETWEEN d.start_date AND d.end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

### Error ❌
#### Here the query looks for start_date and end_date which is not a part of the database

In [12]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('26285.000000'),)]
Answer:26285
> Finished chain.


In [13]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('26285'),)]
Answer:26285
> Finished chain.


In [14]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(90,), (58,), (20,), (41,), (51,)]
Answer:90
> Finished chain.


In [15]:
qns5 = db_chain.run("SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('260'),)]
Answer:260
> Finished chain.


## Few Shot Learning
We will use few shot learning to fix the errors we have faced so far!

In [16]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

## Creating Semantic Similarity Based example selector
#### create embedding on the few_shots
#### Store the embeddings in Chroma DB
#### Retrieve the the top most Semantically close example from the vector store

In [20]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

def ensure_strings(example):
    return " ".join([str(value) for value in example.values()])

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [ensure_strings(example) for example in few_shots]

In [21]:
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query {'query': 'How many t-shirts do we have left for nike in extra small size and white color?', 'result': '34'}",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 17927",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 26285",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our stor

In [23]:
def flatten_metadata(metadata):
    return {k: str(v) if isinstance(v, (dict, list)) else v for k, v in metadata.items()}

metadata = [flatten_metadata(example) for example in few_shots]

In [24]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=metadata)

In [25]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

In [26]:
example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': "{'query': 'How many t-shirts do we have left for nike in extra small size and white color?', 'result': '34'}",
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '260',
  'Question': "How many white color Levi's shirt I have?",
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'}]

In [27]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [28]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [29]:
from langchain.prompts.prompt import PromptTemplate

In [30]:
example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [31]:
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [35]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"]
)

In [38]:
new_chain = SQLDatabaseChain.from_llm(
    llm=llm,
    db=db,
    prompt=few_shot_prompt,
    verbose=True
)

In [44]:
query = "How many white color Levi's shirt I have?"

In [45]:
result = new_chain({'query': query})
print(result)



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:

KeyError: "'query'"

In [46]:
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT sum(price * stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('17927'),)]
Answer:17927
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': '17927'}

In [47]:
new_chain("How much is the price of all white color levi t shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t shirts?
SQLQuery:SELECT sum(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('7449'),)]
Answer:7449
> Finished chain.


{'query': 'How much is the price of all white color levi t shirts?',
 'result': '7449'}

In [48]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('19279.600000'),)]
Answer:19279.6
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '19279.6'}

In [49]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('19897.800000'),)]
Answer:19897.8
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '19897.8'}

In [50]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT sum(price * stock_quantity) FROM t_shirts WHERE brand = 'Van Huesen'
SQLResult: [(Decimal('20960'),)]
Answer:20960
> Finished chain.


'20960'